# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc1e8232e20>
├── 'a' --> tensor([[ 0.7018,  0.6071, -0.9971],
│                   [ 0.2574, -0.4088,  0.2996]])
└── 'x' --> <FastTreeValue 0x7fc1e8232c10>
    └── 'c' --> tensor([[-1.7146e-01, -2.6752e-01, -1.9818e-04,  4.8624e-01],
                        [-4.1051e-01, -4.4588e-01,  8.4460e-01, -1.8590e+00],
                        [ 1.1623e+00,  8.4636e-01, -1.4528e-01, -2.4610e+00]])

In [4]:
t.a

tensor([[ 0.7018,  0.6071, -0.9971],
        [ 0.2574, -0.4088,  0.2996]])

In [5]:
%timeit t.a

64.8 ns ± 0.0608 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc1e8232e20>
├── 'a' --> tensor([[ 0.5250,  0.3378,  0.4931],
│                   [-1.0610,  1.4997, -1.1443]])
└── 'x' --> <FastTreeValue 0x7fc1e8232c10>
    └── 'c' --> tensor([[-1.7146e-01, -2.6752e-01, -1.9818e-04,  4.8624e-01],
                        [-4.1051e-01, -4.4588e-01,  8.4460e-01, -1.8590e+00],
                        [ 1.1623e+00,  8.4636e-01, -1.4528e-01, -2.4610e+00]])

In [7]:
%timeit t.a = new_value

69.3 ns ± 0.0515 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7018,  0.6071, -0.9971],
               [ 0.2574, -0.4088,  0.2996]]),
    x: Batch(
           c: tensor([[-1.7146e-01, -2.6752e-01, -1.9818e-04,  4.8624e-01],
                      [-4.1051e-01, -4.4588e-01,  8.4460e-01, -1.8590e+00],
                      [ 1.1623e+00,  8.4636e-01, -1.4528e-01, -2.4610e+00]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7018,  0.6071, -0.9971],
        [ 0.2574, -0.4088,  0.2996]])

In [11]:
%timeit b.a

58.9 ns ± 0.0541 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.7629,  0.5855,  0.6524],
               [-0.7519,  0.1614,  0.4940]]),
    x: Batch(
           c: tensor([[-1.7146e-01, -2.6752e-01, -1.9818e-04,  4.8624e-01],
                      [-4.1051e-01, -4.4588e-01,  8.4460e-01, -1.8590e+00],
                      [ 1.1623e+00,  8.4636e-01, -1.4528e-01, -2.4610e+00]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.0939 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

849 ns ± 1.71 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 47.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 420 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 739 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc11b5417f0>
├── 'a' --> tensor([[[ 0.7018,  0.6071, -0.9971],
│                    [ 0.2574, -0.4088,  0.2996]],
│           
│                   [[ 0.7018,  0.6071, -0.9971],
│                    [ 0.2574, -0.4088,  0.2996]],
│           
│                   [[ 0.7018,  0.6071, -0.9971],
│                    [ 0.2574, -0.4088,  0.2996]],
│           
│                   [[ 0.7018,  0.6071, -0.9971],
│                    [ 0.2574, -0.4088,  0.2996]],
│           
│                   [[ 0.7018,  0.6071, -0.9971],
│                    [ 0.2574, -0.4088,  0.2996]],
│           
│                   [[ 0.7018,  0.6071, -0.9971],
│                    [ 0.2574, -0.4088,  0.2996]],
│           
│                   [[ 0.7018,  0.6071, -0.9971],
│                    [ 0.2574, -0.4088,  0.2996]],
│           
│                   [[ 0.7018,  0.6071, -0.9971],
│                    [ 0.2574, -0.4088,  0.2996]]])
└── 'x' --> <FastTreeValue 0x7fc11b5412e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.4 µs ± 56.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc1e82292e0>
├── 'a' --> tensor([[ 0.7018,  0.6071, -0.9971],
│                   [ 0.2574, -0.4088,  0.2996],
│                   [ 0.7018,  0.6071, -0.9971],
│                   [ 0.2574, -0.4088,  0.2996],
│                   [ 0.7018,  0.6071, -0.9971],
│                   [ 0.2574, -0.4088,  0.2996],
│                   [ 0.7018,  0.6071, -0.9971],
│                   [ 0.2574, -0.4088,  0.2996],
│                   [ 0.7018,  0.6071, -0.9971],
│                   [ 0.2574, -0.4088,  0.2996],
│                   [ 0.7018,  0.6071, -0.9971],
│                   [ 0.2574, -0.4088,  0.2996],
│                   [ 0.7018,  0.6071, -0.9971],
│                   [ 0.2574, -0.4088,  0.2996],
│                   [ 0.7018,  0.6071, -0.9971],
│                   [ 0.2574, -0.4088,  0.2996]])
└── 'x' --> <FastTreeValue 0x7fc11b8603d0>
    └── 'c' --> tensor([[-1.7146e-01, -2.6752e-01, -1.9818e-04,  4.8624e-01],
                        [-4.1051e-01, -4.4588e-01,  8.4460e-01,

In [23]:
%timeit t_cat(trees)

31.5 µs ± 47.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.5 µs ± 81.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.7018,  0.6071, -0.9971],
                [ 0.2574, -0.4088,  0.2996]],
       
               [[ 0.7018,  0.6071, -0.9971],
                [ 0.2574, -0.4088,  0.2996]],
       
               [[ 0.7018,  0.6071, -0.9971],
                [ 0.2574, -0.4088,  0.2996]],
       
               [[ 0.7018,  0.6071, -0.9971],
                [ 0.2574, -0.4088,  0.2996]],
       
               [[ 0.7018,  0.6071, -0.9971],
                [ 0.2574, -0.4088,  0.2996]],
       
               [[ 0.7018,  0.6071, -0.9971],
                [ 0.2574, -0.4088,  0.2996]],
       
               [[ 0.7018,  0.6071, -0.9971],
                [ 0.2574, -0.4088,  0.2996]],
       
               [[ 0.7018,  0.6071, -0.9971],
                [ 0.2574, -0.4088,  0.2996]]]),
    x: Batch(
           c: tensor([[[-1.7146e-01, -2.6752e-01, -1.9818e-04,  4.8624e-01],
                       [-4.1051e-01, -4.4588e-01,  8.4460e-01, -1.8590e+00],
                       [ 1.1623e+00,  8

In [26]:
%timeit Batch.stack(batches)

80.4 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.7018,  0.6071, -0.9971],
               [ 0.2574, -0.4088,  0.2996],
               [ 0.7018,  0.6071, -0.9971],
               [ 0.2574, -0.4088,  0.2996],
               [ 0.7018,  0.6071, -0.9971],
               [ 0.2574, -0.4088,  0.2996],
               [ 0.7018,  0.6071, -0.9971],
               [ 0.2574, -0.4088,  0.2996],
               [ 0.7018,  0.6071, -0.9971],
               [ 0.2574, -0.4088,  0.2996],
               [ 0.7018,  0.6071, -0.9971],
               [ 0.2574, -0.4088,  0.2996],
               [ 0.7018,  0.6071, -0.9971],
               [ 0.2574, -0.4088,  0.2996],
               [ 0.7018,  0.6071, -0.9971],
               [ 0.2574, -0.4088,  0.2996]]),
    x: Batch(
           c: tensor([[-1.7146e-01, -2.6752e-01, -1.9818e-04,  4.8624e-01],
                      [-4.1051e-01, -4.4588e-01,  8.4460e-01, -1.8590e+00],
                      [ 1.1623e+00,  8.4636e-01, -1.4528e-01, -2.4610e+00],
                      [-1.7146e-01, -2.6752e-

In [28]:
%timeit Batch.cat(batches)

147 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

341 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
